# Install dependecies

In [1]:
!pip install tensorflow==2.4.1 opencv-python matplotlib

You should consider upgrading via the '/Users/sanjeev/Documents/faceid/faceid/bin/python3 -m pip install --upgrade pip' command.


# Import dependencies

In [2]:
#standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#tensorflow dependencies- Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [4]:
#Setup paths
POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data','negative')
ANC_PATH = os.path.join('data','anchor')

In [5]:
#making directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [6]:
#uncompress tar gz labelled faces in the wild dataset
!tar -xf lfw.tgz

In [7]:
#Move lfw images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw',directory)):
        EX_PATH = os.path.join('lfw',directory,file)
        NEW_PATH = os.path.join(NEG_PATH,file)
        os.replace(EX_PATH,NEW_PATH)

In [8]:
#import uuid to generate unique image names
import uuid

In [10]:
#establish a connection to the webcame
cap = cv2.VideoCapture(0)
while cap.isOpened(): #loop through frames till the webcam is open
    ret,frame = cap.read() #read frames at every point in time
    
    frame = frame[130:130+250,530:530+250,:] #cut down frame to 250 X 250
    
    if cv2.waitKey(1) & 0XFF == ord('a'): #collecting anchor images
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())) #create the unique file path
        cv2.imwrite(imgname, frame) #write out anchor image
        
    if cv2.waitKey(1) & 0XFF == ord('p'): #collecting positive images
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    cv2.imshow('Image Collection', frame) #rendering the frame back to the screen
    
    if cv2.waitKey(1) & 0XFF == ord('q'): #breaking
        break
        
cap.release() #release the webcam
cv2.destroyAllWindows() #close the image show frame